In [1]:
from essential_takehome.files import load_dataframes
datasets = load_dataframes()

2023-11-28 11:00:15.300 | INFO     | essential_takehome.files:load_dataframes:31 - Loaded file country_id_mapping.csv
2023-11-28 11:00:15.337 | INFO     | essential_takehome.files:load_dataframes:31 - Loaded file customer_data.csv
2023-11-28 11:00:15.653 | INFO     | essential_takehome.files:load_dataframes:31 - Loaded file session_clickstream_data.csv
2023-11-28 11:00:15.659 | INFO     | essential_takehome.files:load_dataframes:31 - Loaded file sku_data.csv


# Test Agent Output

In [2]:
import pandas as pd

# Step 1: Merge the necessary dataframes
customer_data = datasets['customer_data.csv']
session_clickstream_data = datasets['session_clickstream_data.csv']
sku_data = datasets['sku_data.csv']

merged_data = pd.merge(session_clickstream_data, customer_data, on='Customer ID')
merged_data = pd.merge(merged_data, sku_data, on='SKU ID')

# oh dear this is bad; fanning out the measure then aggregating
# Step 2: Calculate the total purchases for each customer
customer_purchases = merged_data.groupby('Customer ID')['All Purchases'].sum()

# Step 3: Identify top customers with high purchases
top_customers = customer_purchases.nlargest(5)

# Step 4: Determine the common frequency of purchases for top customers
common_frequency = merged_data[merged_data['Customer ID'].isin(top_customers.index)].groupby('Customer ID')['All Purchases'].mean()

print(common_frequency)

Customer ID
8201     161.0
13846    192.0
19623    142.0
22433    195.0
22902    185.0
Name: All Purchases, dtype: float64


# Manually Defined Answer

In [14]:
import pandas as pd

# Step 1: Merge the necessary dataframes
customer_data = datasets['customer_data.csv']
session_clickstream_data = datasets['session_clickstream_data.csv']
sku_data = datasets['sku_data.csv']

customer_data = customer_data.nlargest(5, 'All Purchases')

merged_data = pd.merge(session_clickstream_data, customer_data, on='Customer ID')
aggregated_data = merged_data.groupby('Customer ID').agg(
    All_Purchases=pd.NamedAgg(column='All Purchases', aggfunc='first'),  # Or use 'max' if more appropriate
    Purchased_Count=pd.NamedAgg(column='Purchased', aggfunc=lambda x: x.sum()),
    Unique_Months_Count=pd.NamedAgg(column='month', aggfunc='nunique')  # Assuming 'Month' is a column in your data
)
# merged_data = pd.merge(merged_data, sku_data, on='SKU ID')

unique_months_count = session_clickstream_data['month'].nunique()


aggregated_data['Purchases_Per_Month'] = aggregated_data['Purchased_Count'] / unique_months_count
print(aggregated_data[['All_Purchases', 'Purchases_Per_Month']])

             All_Purchases  Purchases_Per_Month
Customer ID                                    
8201                   161                  0.2
13846                  192                  0.2
19623                  142                  0.2
22433                  195                  0.2
22902                  185                  0.2
